### What is GridSearch?
GridSearch is an optimization tool that we use when tuning hyperparameters. We define the grid of parameters that we want to search through, and we select the best combination of parameters for our data.

https://towardsdatascience.com/gridsearch-the-ultimate-machine-learning-tool-6cd5fb93d07

### The “Grid” in GridSearch

![grid](grid.png)

# 1: One way

In [52]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [1]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()

# 'kernel':('linear', 'poly', 'rbf', 'sigmoid'), 

parameters = {
    'kernel': ('linear', 'rbf', 'sigmoid'), 
    'C':[0.0001,0.1, 0.5, 1, 5, 10, 100], 
    'degree': [1,2,3,4,5,6,7,8,9],
    'coef0': [-10.,-1., 0., 0.1, 0.5, 1, 10, 100],
    'gamma': ('scale', 'auto')
    }

svc = svm.SVC()

clf = GridSearchCV(estimator=svc, param_grid=parameters, n_jobs=-1, cv=10)

X = iris.data
y = iris.target
clf.fit(X, y)

print("clf.best_stimator_", clf.best_estimator_)
print("clf.best_params_", clf.best_params_)
# Mean cross-validated score of the best_estimator
print("clf.best_score", clf.best_score_)

clf.best_stimator_ SVC(C=0.5, coef0=-10.0, degree=1, kernel='linear')
clf.best_params_ {'C': 0.5, 'coef0': -10.0, 'degree': 1, 'gamma': 'scale', 'kernel': 'linear'}
clf.best_score 0.9866666666666667


# 2: Almost-Pro way

La forma pro es la que hace esto mismo y va recogiendo los errores de entrenamiento, de validación y tiene la capacidad de parar el proceso cuando se requiera además de guardar el modelo en local una vez terminado si es mejor que el que había anteriormente y de cargar el modelo anterior y seguir reentrenando.

In [2]:
import pickle

In [3]:
# Load libraries
import numpy as np
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split 
# Set random seed
np.random.seed(0)

In [4]:
# Load data
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [6]:
to_test = np.arange(1, 10)

In [8]:
# Create a pipeline

# Le podemos poner cualquier clasificador. Irá cambiando según va probando pero necesita 1.
pipe = Pipeline(steps=[('classifier', RandomForestClassifier())])


logistic_params = {
    'classifier': [LogisticRegression()],
    'classifier__penalty': ['l1', 'l2'],
    'classifier__C': np.logspace(0, 4, 10)
    }

random_forest_params = {
    'classifier': [RandomForestClassifier()],
    'classifier__n_estimators': [10, 100, 1000],
    'classifier__max_features': [1, 2, 3]
    }

svm_params = {
    'classifier': [svm.SVC()],
    'classifier__kernel':('linear', 'rbf', 'sigmoid'), 
    'classifier__C':[0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 
    'classifier__degree': to_test,
    'classifier__coef0': [-10.,-1., 0., 0.1, 0.5, 1, 10, 100],
    'classifier__gamma': ('scale', 'auto')
    }

# hypertuning 
# Create space of candidate learning algorithms and their hyperparameters
search_space = [
    logistic_params,
    random_forest_params,
    svm_params
    ]

In [13]:
%%time

cv = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
# Create grid search 
clf = GridSearchCV(estimator=pipe, param_grid=search_space, cv=cv, verbose=1, n_jobs=-1)

# Fit grid search
best_model = clf.fit(X_train, y_train)

# View best model
separator = "\n############################\n"
print(separator)
svc_ganador_entrado = best_model.best_estimator_.get_params()['classifier']
print("best estimator:", best_model.best_estimator_.get_params()['classifier'])
print(separator)
print("clf.best_params_", clf.best_params_)
print(separator)
# Mean cross-validated score of the best_estimator
print("clf.best_score", clf.best_score_)
#SAVE MODEL
# save the model to disk
filename = 'finished_model.sav'
pickle.dump(best_model, open(filename, 'wb'))

Fitting 10 folds for each of 3485 candidates, totalling 34850 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 7160 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 22960 tasks      | elapsed:   13.4s

############################

best estimator: SVC(C=0.5, coef0=-10.0, degree=1, kernel='linear')

############################

clf.best_params_ {'classifier': SVC(C=0.5, coef0=-10.0, degree=1, kernel='linear'), 'classifier__C': 0.5, 'classifier__coef0': -10.0, 'classifier__degree': 1, 'classifier__gamma': 'scale', 'classifier__kernel': 'linear'}

############################

clf.best_score 0.9916666666666666
Wall time: 18.5 s
[Parallel(n_jobs=-1)]: Done 34835 out of 34850 | elapsed:   18.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 34850 out of 34850 | elapsed:   18.3s finished


In [11]:
path = os.getcwd() + os.sep
full_file_name = path + "finished_model.sav"
loaded_model = pickle.load(open(full_file_name, 'rb'))

In [12]:
# Predict target vector
best_model.score(X_test, y_test) * 100

100.0

In [14]:
type(svc_ganador_entrado)

sklearn.svm._classes.SVC

In [15]:
type(loaded_model)

sklearn.model_selection._search.GridSearchCV

# 3 Another way - No pro

In [27]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

# Loading the Digits dataset
digits = datasets.load_digits()

# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target

In [28]:
# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 
                    'gamma': [1e-3, 1e-4], 
                    'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 
                    'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

In [29]:
bar = "######################################"
for score in scores:
    print(bar + "\n########## SCORE " + score + " ########### \n" + bar)
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(estimator=SVC(), param_grid=tuned_parameters, scoring=str(score)+'_macro')
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

######################################
########## SCORE precision ########### 
######################################
# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.986 (+/-0.016) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.959 (+/-0.028) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.988 (+/-0.017) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.982 (+/-0.026) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.988 (+/-0.017) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.983 (+/-0.026) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.988 (+/-0.017) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.983 (+/-0.026) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.974 (+/-0.012) for {'C': 1, 'kernel': 'linear'}
0.974 (+/-0.012) for {'C': 10, 'kernel': 'linear'}
0.974 (+/-0.012) for {'C': 100, 'kernel': 'linear'}
0.974 (+/-0.012) for {'C': 1000, 'kernel': '